# Introductory H2O Machine Learning Tutorial

Prepared for H2O Open Chicago 2016: http://open.h2o.ai/chicago.html

## Install H2O

The first step in this tutorial is to download and install the h2o Python module.  
The latest version is always here: http://www.h2o.ai/download/h2o/py

### Start up the H2O Cluster

Once the Python module is installed, we begin by starting up a local (on your laptop) H2O cluster.

In [1]:
# Load the H2O library and start up the H2O cluter locally on your machine
import h2o

# Number of threads, nthreads = -1, means use all cores on your machine
# max_mem_size is the maximum memory (in GB) to allocate to H2O
h2o.init(nthreads = -1, max_mem_size = 8)



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /var/folders/2j/jg4sl53d5q53tc2_nzm9fz5h0000gn/T/tmp2QMc58/h2o_me_started_from_python.out
JVM stderr: /var/folders/2j/jg4sl53d5q53tc2_nzm9fz5h0000gn/T/tmpgB7EFj/h2o_me_started_from_python.err
Using ice_root: /var/folders/2j/jg4sl53d5q53tc2_nzm9fz5h0000gn/T/tmp2jVpCf


Java Version: java version "1.8.0_45"
Java(TM) SE Runtime Environment (build 1.8.0_45-b14)
Java HotSpot(TM) 64-Bit Server VM (build 25.45-b02, mixed mode)


Starting H2O JVM and connecting: ......... Connection successful!


/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

H2O cluster uptime:,1 seconds 34 milliseconds
H2O cluster version:,3.8.2.3
H2O cluster name:,H2O_started_from_python_me_tva596
H2O cluster total nodes:,1
H2O cluster total free memory:,7.11 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


## Data prep

### Import data
Next we will import a cleaned up version of the Lending Club "Bad Loans" dataset. The purpose here is to predict whether a loan will be bad (i.e. not repaid to the lender). The response column, `bad_loan`, is 1 if the loan was bad, and 0 otherwise.

In [2]:
loan_csv = "/Volumes/H2OTOUR/loan.csv"  # modify this for your machine
# Alternatively, you can import the data directly from a URL
#loan_csv = "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"
data = h2o.import_file(loan_csv)  # 163,987 rows x 15 columns


Parse Progress: [                                                  ] 00%

/usr/local/lib/python2.7/site-packages/h2o/h2o.py:316: UserWarning: ParseError at file nfs://Volumes/H2OTOUR/loan.csv  at byte offset 4194304; error = 'Unmatched quote char "'
  warnings.warn(w)


Parse Progress: [##################################################] 100%


In [3]:
data.shape

(163987, 15)

### Encode response variable
Since we want to train a binary classification model, we must ensure that the response is coded as a factor. If the response is 0/1, H2O will assume it's numeric, which means that H2O will train a regression model instead.

In [4]:
data['bad_loan'] = data['bad_loan'].asfactor()  #encode the binary repsonse as a factor
data['bad_loan'].levels()  #optional: after encoding, this shows the two factor levels, '0' and '1'

[['0', '1']]

### Partition data

Next, we partition the data into training, validation and test sets.

In [5]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

Notice that `split_frame()` uses approximate splitting not exact splitting (for efficiency), so these are not exactly 70%, 15% and 15% of the total rows.

In [6]:
print train.nrow
print valid.nrow
print test.nrow

114908
24498
24581


### Identify response and predictor variables
In H2O, we use `y` to designate the response variable and `x` to designate the list of predictor columns.

In [7]:
y = 'bad_loan'
x = list(data.columns)

In [8]:
x.remove(y)  #remove the response
x.remove('int_rate')  #remove the interest rate column because it's correlated with the outcome

In [9]:
# List of predictor columns
x

[u'loan_amnt',
 u'term',
 u'emp_length',
 u'home_ownership',
 u'annual_inc',
 u'purpose',
 u'addr_state',
 u'dti',
 u'delinq_2yrs',
 u'revol_util',
 u'total_acc',
 u'longest_credit_length',
 u'verification_status']

## H2O Machine Learning

Now that we have prepared the data, we can train some models. We will start by training a single model from each of the H2O supervised algos:

- Generalized Linear Model (GLM)
- Random Forest (RF)
- Gradient Boosting Machine (RF)
- Deep Learning (DL)
- Naive Bayes (NB)

## 1. Generalized Linear Model
Let's start with a basic binomial Generalized Linear Model (GLM).  By default, H2O's GLM uses a regularized, elastic net model.

In [10]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

### Train a default GLM
We first create an object of class, `"H2OGeneralizedLinearEstimator"`.  This does not actually do any training, it just sets the model up for training by specifying model parameters.

In [11]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

Now that `glm_fit1` object is initialized, we can train the model:

In [12]:
glm_fit1.train(x=x, y=y, training_frame=train)


glm Model Build Progress: [##################################################] 100%


### Train a GLM with lambda search

Next we will do some automatic tuning by passing in a validation frame and setting `lambda_search = True`.  Since we are training a GLM with regularization, we should try to find the right amount of regularization (to avoid overfitting).  The model parameter, `lambda`, controls the amount of regularization in a GLM model and we can find the optimal value for `lambda` automatically by setting `lambda_search = True` and passing in a validation frame (which is used to evaluate model performance using a particular value of lambda).

In [13]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit2', lambda_search=True)
glm_fit2.train(x=x, y=y, training_frame=train, validation_frame=valid)


glm Model Build Progress: [##################################################] 100%


### Evaluate model performance
Let's compare the performance of the two GLMs that were just trained.

In [14]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)

In [15]:
# Print model performance
print glm_perf1
print glm_perf2


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.14215506685
R^2: 0.0639172702162
LogLoss: 0.451078449301
Null degrees of freedom: 24580
Residual degrees of freedom: 24529
Null deviance: 23672.9222656
Residual deviance: 22175.9187245
AIC: 22279.9187245
AUC: 0.677449084114
Gini: 0.354898168228

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.193355175264: 


,0,1,Error,Rate
0,13646.0,6345.0,0.3174,(6345.0/19991.0)
1,1939.0,2651.0,0.4224,(1939.0/4590.0)
Total,15585.0,8996.0,0.337,(8284.0/24581.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1933552,0.3902547,225.0
max f2,0.1187713,0.5566546,306.0
max f0point5,0.2776294,0.3538608,149.0
max accuracy,0.4942391,0.8144095,32.0
max precision,0.7445071,1.0,0.0
max recall,0.0025754,1.0,398.0
max specificity,0.7445071,1.0,0.0
max absolute_MCC,0.1984005,0.2106992,220.0
max min_per_class_accuracy,0.1800948,0.6279826,238.0



Gains/Lift Table: Avg response rate: 18.67 %



...

,0,1,Error,Rate
0,13596.0,6395.0,0.3199,(6395.0/19991.0)
1,1928.0,2662.0,0.42,(1928.0/4590.0)
Total,15524.0,9057.0,0.3386,(8323.0/24581.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1929457,0.3901224,220.0
max f2,0.1189482,0.5566554,306.0
max f0point5,0.2744616,0.3542416,146.0
max accuracy,0.4965808,0.8144095,29.0
max precision,0.7454046,1.0,0.0
max recall,0.0027638,1.0,398.0
max specificity,0.7454046,1.0,0.0
max absolute_MCC,0.1982820,0.2105847,215.0
max min_per_class_accuracy,0.1802337,0.6298475,234.0



Gains/Lift Table: Avg response rate: 18.67 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100077,0.4655732,2.8953655,2.8953655,0.5406504,0.5406504,0.0289760,0.0289760,189.5365499,189.5365499
,2,0.0200155,0.4293011,2.3946632,2.6450143,0.4471545,0.4939024,0.0239651,0.0529412,139.4663195,164.5014347
,3,0.0300232,0.4025228,2.2640452,2.5180246,0.4227642,0.4701897,0.0226580,0.0755991,126.4045203,151.8024632
,4,0.0400309,0.3850863,2.2640452,2.4545298,0.4227642,0.4583333,0.0226580,0.0982571,126.4045203,145.4529775
,5,0.0500386,0.3697433,1.9592699,2.3554778,0.3658537,0.4398374,0.0196078,0.1178649,95.9269887,135.5477797
,6,0.1000366,0.3099855,1.9085744,2.1321170,0.3563873,0.3981293,0.0954248,0.2132898,90.8574376,113.2116958
,7,0.1500346,0.2745911,1.6994155,1.9879223,0.3173312,0.3712039,0.0849673,0.2982571,69.9415541,98.7922261
,8,0.2000325,0.2483325,1.4118221,1.8439265,0.2636290,0.3443156,0.0705882,0.3688453,41.1822141,84.3926522
,9,0.3000285,0.2108798,1.2745617,1.6541640,0.2379984,0.3088814,0.1274510,0.4962963,27.4561655,65.4163967
,10,0.4000244,0.1845418,1.1525523,1.5287738,0.2152156,0.2854673,0.1152505,0.6115468,15.2552335,52.8773812


Instead of printing the entire model performance metrics object, it is probably easier to print just the metric that you are interested in comparing.

In [16]:
# Retreive test set AUC
print glm_perf1.auc()
print glm_perf2.auc()

0.677449084114
0.677675858276


In [17]:
# Compare test AUC to the training AUC and validation AUC
print glm_fit2.auc(train=True)
print glm_fit2.auc(valid=True)

0.674306164325
0.675512216705


## 2. Random Forest
H2O's Random Forest (RF) is implements a distributed version of the standard Random Forest algorithm and variable importance measures.

In [18]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

### Train a default RF
First we will train a basic Random Forest model with default parameters. Random Forest will infer the response distribution from the response encoding. A seed is required for reproducibility.

In [19]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

Now that `rf_fit1` object is initialized, we can train the model:

In [20]:
rf_fit1.train(x=x, y=y, training_frame=train)


drf Model Build Progress: [##################################################] 100%


### Train an RF with more trees

Next we will increase the number of trees used in the forest by setting `ntrees = 100`.  The default number of trees in an H2O Random Forest is 50, so this RF will be twice as big as the default.  Usually increasing the number of trees in an RF will increase performance as well.  Unlike Gradient Boosting Machines (GBMs), Random Forests are fairly resistant (although not free from) overfitting by increasing the number of trees.  See the GBM example below for additional guidance on preventing overfitting using H2O's early stopping functionality.

In [21]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)


drf Model Build Progress: [##################################################] 100%


### Compare model performance
Let's compare the performance of the two RFs that were just trained.

In [22]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [23]:
# Retreive test set AUC
print rf_perf1.auc()
print rf_perf2.auc()

0.662266990734
0.66525468051


### Cross-validate performance

Rather than using held-out test set to evaluate model performance, a user may wish to estimate model performance using cross-validation.  Using the RF algorithm (with default model parameters) as an example, we demonstrate how to perform k-fold cross-validation using H2O.  No custom code or loops are required, you simply specify the number of desired folds in the `nfolds` argument.

Since we are not going to use a test set here, we can use the original (full) dataset, which we called `data` rather than the subsampled `train` dataset.  Note that this will take approximately k (`nfolds`) times longer than training a single RF model, since it will train k models in the cross-validation process (trained on n(k-1)/k rows), in addition to the final model trained on the full `training_frame` dataset with n rows. 

In [24]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5)
rf_fit3.train(x=x, y=y, training_frame=data)


drf Model Build Progress: [##################################################] 100%


To evaluate the cross-validated AUC, do the following:

In [25]:
print rf_fit3.auc(xval=True)

0.661201482614


Note that the cross-validated AUC is slighly higher than the test set performance we estimated for `rf_fit1`, and this is likely due to the fact that we trained on more data (n rows) than we did while using `train` as the training set (0.75*n rows) in `rf_fit1`.

## 3. Gradient Boosting Machine
H2O's Gradient Boosting Machine (GBM) offers a Stochastic GBM, which can increase performance quite a bit compared to the original GBM implementation.

In [26]:
# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

### Train a default GBM

First we will train a basic GBM model with default parameters. GBM will infer the response distribution from the response encoding if not specified explicitly through the `distribution` argument. A seed is required for reproducibility.

In [27]:
# Initialize and train the GBM estimator:

gbm_fit1 = H2OGradientBoostingEstimator(model_id='gbm_fit1', seed=1)
gbm_fit1.train(x=x, y=y, training_frame=train)


gbm Model Build Progress: [##################################################] 100%


### Train a GBM with more trees

Next we will increase the number of trees used in the GBM by setting `ntrees=500`.  The default number of trees in an H2O GBM is 50, so this GBM will trained using ten times the default.  Increasing the number of trees in a GBM is one way to increase performance of the model, however, you have to be careful not to overfit your model to the training data by using too many trees.  To automatically find the optimal number of trees, you must use H2O's early stopping functionality.  This example will not do that, however, the following example will.

In [28]:
gbm_fit2 = H2OGradientBoostingEstimator(model_id='gbm_fit2', ntrees=500, seed=1)
gbm_fit2.train(x=x, y=y, training_frame=train)


gbm Model Build Progress: [##################################################] 100%


### Train a GBM with early stopping

We will again set `ntrees = 500`, however, this time we will use early stopping in order to prevent overfitting (from too many trees).  All of H2O's algorithms have early stopping available, however, with the exception of Deep Learning, it is not enabled by default.  

There are several parameters that should be used to control early stopping.  The three that are generic to all the algorithms are: `stopping_rounds`, `stopping_metric` and `stopping_tolerance`.  The stopping metric is the metric by which you'd like to measure performance, and so we will choose AUC here.  The `score_tree_interval` is a parameter specific to Random Forest and GBM.  Setting `score_tree_interval=5` will score the model after every five trees.  The parameters we have set below specify that the model will stop training after there have been three scoring intervals where the AUC has not increased more than 0.0005.  Since we have specified a validation frame, the stopping tolerance will be computed on validation AUC rather than training AUC. 

In [29]:
# Now let's use early stopping to find optimal ntrees

gbm_fit3 = H2OGradientBoostingEstimator(model_id='gbm_fit3', 
                                        ntrees=500, 
                                        score_tree_interval=5,     #used for early stopping
                                        stopping_rounds=3,         #used for early stopping
                                        stopping_metric='AUC',     #used for early stopping
                                        stopping_tolerance=0.0005, #used for early stopping
                                        seed=1)

# The use of a validation_frame is recommended with using early stopping
gbm_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)


gbm Model Build Progress: [##################################################] 100%


### Compare model performance

Let's compare the performance of the three GBMs that were just trained.

In [30]:
gbm_perf1 = gbm_fit1.model_performance(test)
gbm_perf2 = gbm_fit2.model_performance(test)
gbm_perf3 = gbm_fit3.model_performance(test)

In [31]:
# Retreive test set AUC
print gbm_perf1.auc()
print gbm_perf2.auc()
print gbm_perf3.auc()

0.682765594191
0.671854616713
0.68309902855


### Scoring History

To examine the scoring history, use the `scoring_history` method on a trained model.  If `score_tree_interval` is not specified, it will score at various intervals, as we can see for `gbm_fit2.scoring_history()` below.  However, regular 5-tree intervals are used for `gbm_fit3.scoring_history()`.  

The `gbm_fit2` was trained only using a training set (no validation set), so the scoring history is calculated for training set performance metrics only.

In [47]:
gbm_fit2.scoring_history()

,,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
0,,2016-05-03 10:49:35,0.001 sec,0,0.148714,0.474030,0.500000,1.000000,0.818255
1,,2016-05-03 10:49:35,0.079 sec,1,0.147252,0.469209,0.657210,2.680144,0.400599
2,,2016-05-03 10:49:35,0.142 sec,2,0.146021,0.465266,0.664849,2.663689,0.346703
3,,2016-05-03 10:49:36,0.208 sec,3,0.144996,0.462036,0.667746,2.860431,0.339689
4,,2016-05-03 10:49:36,0.288 sec,4,0.144123,0.459326,0.669405,2.900994,0.361158
5,,2016-05-03 10:49:36,0.390 sec,5,0.143373,0.456995,0.672514,3.013793,0.338366
6,,2016-05-03 10:49:36,0.547 sec,6,0.142734,0.455020,0.674697,3.080660,0.352987
7,,2016-05-03 10:49:36,0.731 sec,7,0.142280,0.453580,0.675089,3.088117,0.336130
8,,2016-05-03 10:49:36,0.961 sec,8,0.141908,0.452386,0.675682,3.056254,0.341943
9,,2016-05-03 10:49:37,1.195 sec,9,0.141588,0.451341,0.676213,3.064215,0.330917


When early stopping is used, we see that training stopped at 105 trees instead of the full 500.  Since we used a validation set in `gbm_fit3`, both training and validation performance metrics are stored in the scoring history object.  Take a look at the validation AUC to observe that the correct stopping tolerance was enforced.

In [48]:
gbm_fit3.scoring_history()

,,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error,validation_MSE,validation_logloss,validation_AUC,validation_lift,validation_classification_error
0,,2016-05-03 10:50:06,0.001 sec,0,0.148714,0.474030,0.500000,1.000000,0.818255,0.151042,0.479533,0.500000,1.000000,0.814597
1,,2016-05-03 10:50:07,0.252 sec,5,0.143373,0.456995,0.672514,3.013793,0.338366,0.146227,0.464094,0.661129,2.618090,0.365377
2,,2016-05-03 10:50:07,0.588 sec,10,0.141283,0.450355,0.677305,3.071435,0.326792,0.144531,0.458612,0.663678,2.729585,0.376765
3,,2016-05-03 10:50:08,1.056 sec,15,0.139958,0.446166,0.684262,3.202491,0.342204,0.143614,0.455618,0.667416,2.795897,0.365907
4,,2016-05-03 10:50:08,1.584 sec,20,0.138783,0.442520,0.691469,3.330029,0.347696,0.142869,0.453261,0.671361,2.927986,0.348600
5,,2016-05-03 10:50:09,2.179 sec,25,0.137810,0.439517,0.697321,3.420935,0.301302,0.142248,0.451328,0.674848,2.861942,0.345130
6,,2016-05-03 10:50:09,2.820 sec,30,0.137079,0.437235,0.701716,3.507056,0.304139,0.141892,0.450176,0.676858,2.927986,0.359172
7,,2016-05-03 10:50:10,3.522 sec,35,0.136501,0.435430,0.705013,3.502272,0.313198,0.141634,0.449319,0.678392,2.972016,0.368520
8,,2016-05-03 10:50:11,4.292 sec,40,0.135956,0.433766,0.708290,3.583609,0.309265,0.141474,0.448720,0.679649,2.905971,0.369214
9,,2016-05-03 10:50:12,5.093 sec,45,0.135520,0.432420,0.710795,3.588393,0.301250,0.141333,0.448253,0.680504,3.038061,0.355090


## 4. Deep Learning

H2O's Deep Learning algorithm is a multilayer feed-forward artificial neural network.  It can also be used to train an autoencoder, however, in the example below we will train a standard supervised prediction model.

In [49]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

### Train a default DL

First we will train a basic DL model with default parameters. DL will infer the response distribution from the response encoding if not specified explicitly through the `distribution` argument.  H2O's DL will not be reproducbible if run on more than a single core, so in this example, the performance metrics below may vary slightly from what you see on your machine.

In H2O's DL, early stopping is enabled by default, so below, it will use the training set and default stopping parameters to perform early stopping.

In [51]:
# Initialize and train the DL estimator:

dl_fit1 = H2ODeepLearningEstimator(model_id='dl_fit1', seed=1)
dl_fit1.train(x=x, y=y, training_frame=train)

H2O cluster uptime:,16 minutes 43 seconds 380 milliseconds
H2O cluster version:,3.8.2.3
H2O cluster name:,H2O_started_from_python_me_wzy124
H2O cluster total nodes:,1
H2O cluster total free memory:,7.03 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None



deeplearning Model Build Progress: [##################################################] 100%


### Train a DL with new architecture and more epochs

Next we will increase the number of epochs used in the GBM by setting `epochs=20` (the default is 10).  Increasing the number of epochs in a deep neural net may increase performance of the model, however, you have to be careful not to overfit your model.  To automatically find the optimal number of epochs, you must use H2O's early stopping functionality.  Unlike the rest of the H2O algorithms, H2O's DL will use early by default, so we will first turn it off in the next example by setting `stopping_rounds=0`, for comparison.

In [36]:
dl_fit2 = H2ODeepLearningEstimator(model_id='dl_fit2', 
                                   epochs=20, 
                                   hidden=[10,10], 
                                   stopping_rounds=0,  #disable early stopping
                                   seed=1)
dl_fit2.train(x=x, y=y, training_frame=train)


deeplearning Model Build Progress: [##################################################] 100%


### Train a DL with early stopping

This example will use the same model parameters as `dl_fit2`, however, we will turn on early stopping and specify the stopping criterion.  We will also pass a validation set, as is recommended for early stopping.

In [37]:
dl_fit3 = H2ODeepLearningEstimator(model_id='dl_fit3', 
                                   epochs=20, 
                                   hidden=[10,10],
                                   score_interval=1,          #used for early stopping
                                   stopping_rounds=3,         #used for early stopping
                                   stopping_metric='AUC',     #used for early stopping
                                   stopping_tolerance=0.0005, #used for early stopping
                                   seed=1)
dl_fit3.train(x=x, y=y, training_frame=train, validation_frame=valid)


deeplearning Model Build Progress: [##################################################] 100%


### Compare model performance

Again, we will compare the model performance of the three models using a test set and AUC.

In [38]:
dl_perf1 = dl_fit1.model_performance(test)
dl_perf2 = dl_fit2.model_performance(test)
dl_perf3 = dl_fit3.model_performance(test)

In [39]:
# Retreive test set AUC
print dl_perf1.auc()
print dl_perf2.auc()
print dl_perf3.auc()

0.679670061767
0.675026229123
0.681501550425


In [40]:
dl_fit3.scoring_history()

,,timestamp,duration,training_speed,epochs,iterations,samples,training_MSE,training_r2,training_logloss,training_AUC,training_lift,training_classification_error,validation_MSE,validation_r2,validation_logloss,validation_AUC,validation_lift,validation_classification_error
0,,2016-05-03 10:51:34,0.000 sec,None,0.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2016-05-03 10:51:35,0.288 sec,518093 rows/sec,0.870192,1,99992,0.143602,0.049866,0.457001,0.665048,2.558203,0.351841,0.143185,0.051933,0.455546,0.668156,2.663807,0.345743
2,,2016-05-03 10:51:36,1.315 sec,607058 rows/sec,6.091290,7,699938,0.143460,0.050810,0.456577,0.667244,2.721493,0.363875,0.142990,0.053225,0.455240,0.671613,2.663807,0.358356
3,,2016-05-03 10:51:37,2.338 sec,662289 rows/sec,12.184347,14,1400079,0.150780,0.002375,0.486599,0.674257,2.667063,0.408879,0.150563,0.003085,0.486139,0.677053,2.311568,0.363785
4,,2016-05-03 10:51:38,3.433 sec,698969 rows/sec,19.148841,22,2200355,0.144177,0.046065,0.455084,0.676124,2.939212,0.344357,0.144628,0.042382,0.456287,0.676181,2.641792,0.357376
5,,2016-05-03 10:51:38,3.638 sec,700242 rows/sec,20.018589,23,2300296,0.143916,0.047794,0.460532,0.676903,3.048072,0.375607,0.144099,0.045880,0.461880,0.675448,2.729852,0.340926
6,,2016-05-03 10:51:38,3.710 sec,700029 rows/sec,20.018589,23,2300296,0.150780,0.002375,0.486599,0.674257,2.667063,0.408879,0.150563,0.003085,0.486139,0.677053,2.311568,0.363785


## 4. Naive Bayes

The Naive Bayes (NB) algorithm does not usually beat an algorithm like a Random Forest or GBM, however it is still a popular algorithm, especially in the text domain (when your input is text encoded as "Bag of Words", for example).  The Naive Bayes algorithm is for binary or multiclass classification problems only, not regression.  Therefore, your response must be a factor instead of numeric. 

In [41]:
# Import H2O NB:
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator

### Train a default NB

First we will train a basic NB model with default parameters. 

In [42]:
# Initialize and train the NB estimator:

nb_fit1 = H2ONaiveBayesEstimator(model_id='nb_fit1')
nb_fit1.train(x=x, y=y, training_frame=train)


naivebayes Model Build Progress: [##################################################] 100%


### Train a NB model with Laplace Smoothing

One of the few tunable model parameters for the Naive Bayes algorithm is the amount of Laplace smoothing.  The H2O Naive Bayes model will not use any Laplace smoothing by default.

In [43]:
nb_fit2 = H2ONaiveBayesEstimator(model_id='nb_fit2', laplace=6)
nb_fit2.train(x=x, y=y, training_frame=train)


naivebayes Model Build Progress: [##################################################] 100%


### Compare model performance

We will compare the model performance of the two NB models using test set AUC.

In [44]:
nb_perf1 = nb_fit1.model_performance(test)
nb_perf2 = nb_fit2.model_performance(test)

In [45]:
# Retreive test set AUC
print nb_perf1.auc()
print nb_perf2.auc()

0.651356759779
0.651421140603


In [46]:
h2o.shutdown(prompt=False)